# AuxTel Calibration Illumination System Functional Test

This notebook is meant to be used a functional checkout of the AuxTel illumination system. It includes all functionality but doesn't necessarily show how the system should be used in operation. It was written within the context of the Tucson Teststand and should be modified to be run on the summit.

In [ ]:
import asyncio

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.io import fits
from astropy.time import Time
from astropy.table import Table

from lsst_efd_client import EfdClient
client = EfdClient('tucson_teststand_efd')

from lsst.ts import salobj

In [ ]:
d = salobj.Domain()

### Connect to CSCs

In [ ]:
atmonochromator = salobj.Remote(d, 'ATMonochromator')
FiberSpectrograph = salobj.Remote(name="FiberSpectrograph", domain=d, index=3)
electrometer = salobj.Remote(name="Electrometer", domain=d, index=5) 
WhiteLightSource = salobj.Remote(name='ATWhiteLight', domain=d)

In [ ]:
await atmonochromator.start_task
await FiberSpectrograph.start_task
await electrometer.start_task
await WhiteLightSource.start_task

In [ ]:
async def get_status():
    mono_tmp = await atmonochromator.evt_summaryState.aget()
    spec_tmp = await FiberSpectrograph.evt_summaryState.aget()
    elec_tmp = await electrometer.evt_summaryState.aget()
    wls_tmp = await WhiteLightSource.evt_summaryState.aget()
    print('Monochromator: ',salobj.State(mono_tmp.summaryState))
    print('Fiber Spectrometer: ',salobj.State(spec_tmp.summaryState))
    print('Electrometer: ',salobj.State(elec_tmp.summaryState))
    print('WhiteLightSource: ',salobj.State(wls_tmp.summaryState))

In [ ]:
await get_status()

### Test White Light Source

Note: don't cycle the WLS between Standby and Enabled. You will have to wait 15 minutes on either side of turning it on and off. You can power cycle the other CSCs

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(WhiteLightSource, state)

In [ ]:
#Start chiller
tmp = await WhiteLightSource.cmd_setChillerTemperature.set_start(temperature=20)
tmp = await WhiteLightSource.cmd_startChiller.set_start()
# Confirm that the chiller has started and running at 20C

In [ ]:
#Test shutters
tmp = await WhiteLightSource.cmd_closeShutter.set_start()
tmp = await WhiteLightSource.cmd_openShutter.set_start()
#Keep shutter open so you can see the lamp turned on

In [ ]:
#Turn on lamp. It will then go into a warm up period before it will turn on 
tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power = 800)

In [ ]:
LampBasicState = {0:'Unknown',1:'Off',2:'On',3:'?',4:'Cooldown',5:'Warmup'}
LampControllerError = {-1:'NoError',0:'Unknown',1:'KillSwitch',2:'ChassisOverheating',
                       3:'AccessDoor',4:'BallastOverheasting',5:'USBDisconnected',6:'AirflowMalfunction',
                       7:'LampStuckOn',8:'AirflowMalfunction'}
LampControllerState = {0:'Unknown',1:'Off',2:'StandbyOrOn',3:'Cooldown',4:'Error'}

In [ ]:
state = await WhiteLightSource.evt_lampState.aget()
print(state)
print(LampBasicState[state.basicState], LampControllerError[state.controllerError], LampControllerState[state.controllerState])
print((state.warmupEndTime - state.private_rcvStamp)/60.)
if LampBasicState[state.basicState] == 'Cooldown':
    time_left = state.cooldownEndTime - state.private_rcvStamp
    print('Time Left: {} min.'.format(time_left/60.))

elif LampBasicState[state.basicState] == 'Warmup':
    time_left = state.warmupEndTime - state.private_rcvStamp
    print('Time Left: {} min.'.format(time_left/60.))

In [ ]:
# When it is done warming up, turn up the lamp
tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=1000)

### Test Monochromator

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(atmonochromator, state)

In [ ]:
async def get_params():
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

In [ ]:
await get_params()

In [ ]:
# Change wavelength. Confirm that the color changes
for wave in [400, 500, 600, 700]:
    await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)

In [ ]:
# Change slit size. Confirm that the brightness changes by eye
for entry in [0.5, 4.5]:
    for exit in [0.5, 4.5]:
        await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
        await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)

In [ ]:
# Change grating. Might not see any difference
for grating in [0,1]:
    await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=60)

### Test Electrometer

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(electrometer, state, override='tts_cimacs3_v3.yaml',timeout=20)

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)    

In [ ]:
# Check settings

In [ ]:
exp_time = 1 #sec
etmp1=await electrometer.cmd_startScan.set_start(timeout=10)
await asyncio.sleep(exp_time)
tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)

In [ ]:
# Get data from the efd
start_log_topic = 'lsst.sal.Electrometer.command_startScan'
start_df =  await client.select_time_series(start_log_topic, ['salIndex'], start=start_time, end=end_time)
start_df['message'] = 'startScan Completed'
start_df['functionName'] = 'do_startScan'
msg_log_topic = 'lsst.sal.Electrometer.logevent_logMessage'
msg_df = await client.select_time_series(msg_log_topic,['salIndex','functionName','message'], start=start_time, end=end_time)
elec_df = pd.concat([start_df, msg_df])
elec_df.sort_index(inplace=True)
print(elec_df)

In [ ]:
# Look at data from file
filen = #get from efd


### Test Fiber Spectrograph

In [ ]:
state = salobj.State.ENABLED
tmp = await salobj.set_summary_state(FiberSpectrograph, state)

In [ ]:
exp_time = 1 #sec
FiberSpectrograph.evt_largeFileObjectAvailable.flush()
tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename=lfa.url.split('FiberSpectrograph')[-1]
print(filename)

In [ ]:
#Get data
os.system(f"curl {lfa.url} --output /home/parfa30/DATA/fiberSpec_files/{filename}")
return filename

## Test all Components Together

In [ ]:
async def set_params(wave, entry_width, exit_width, grating):
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=1, slitWidth=entry_width)
    await atmonochromator.cmd_changeSlitWidth.set_start(slit=2, slitWidth=exit_width)
    await atmonochromator.cmd_changeWavelength.set_start(wavelength=wave)
    await atmonochromator.cmd_selectGrating.set_start(gratingType=grating, timeout=60)
   
    tmp1 = await atmonochromator.evt_wavelength.aget()
    tmp2 = await atmonochromator.evt_entrySlitWidth.aget()
    tmp3 = await atmonochromator.evt_exitSlitWidth.aget()
    tmp4 = await atmonochromator.evt_selectedGrating.aget()
    return (tmp1.wavelength, tmp2.width, tmp3.width, tmp4.gratingType)

In [ ]:
async def elec_meas(exp_time):
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)

In [ ]:
async def spect_meas(exp_time):
    FiberSpectrograph.evt_largeFileObjectAvailable.flush()
    tmp1 = await FiberSpectrograph.cmd_expose.set_start(duration=exp_time, numExposures=1)
    lfa = await FiberSpectrograph.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename=lfa.url.split('FiberSpectrograph')[-1]
    os.system(f"curl {lfa.url} --output /home/parfa30/DATA/fiberSpec_files/{filename}")
    return filename

In [ ]:
async def get_elec_data(start_time, end_time, electrometer_id):
    start_log_topic = 'lsst.sal.Electrometer.command_startScan'
    start_df =  await client.select_time_series(start_log_topic, ['salIndex'], start=start_time, end=end_time)
    start_df['message'] = 'startScan Completed'
    start_df['functionName'] = 'do_startScan'
    msg_log_topic = 'lsst.sal.Electrometer.logevent_logMessage'
    msg_df = await client.select_time_series(msg_log_topic,['salIndex','functionName','message'], start=start_time, end=end_time)
    elec_df = pd.concat([start_df, msg_df])
    elec_df.sort_index(inplace=True)
    #print(elec_df)
    data = []
    scans = elec_df[elec_df.salIndex == electrometer_id]
    scans.reset_index(inplace=True)
    if len(scans) > 0:
        for i, row in scans.iterrows():
            if row['functionName'] == 'write_fits_file':
                try:
                    file_row = scans.iloc[i]
                    filen = file_row['message'].split(' ')[-1]
                    summary_row = scans.iloc[i+1]
                    if 'Scan Summary' in summary_row['message']:
                        x = summary_row['message']
                        x = x.split(':')[1].split(',')
                        mean_ = float(x[0].strip(' ').strip('['))
                        median_ = float(x[1].strip(' '))
                        std_ = float(x[2].strip(' ').strip(']'))
                    else:
                        filen, mean_, median_, std_ = 'None', np.nan, np.nan, np.nan
                except:
                    filen, mean_, median_, std_= 'None', np.nan, np.nan, np.nan

                return filen, mean_, median_, std_
    else:
        return ('None', np.nan, np.nan, np.nan)

In [ ]:
data = []
exp_time = 1.
start = Time.now()
for wls_power in [800, 900, 1000, 1100]:
    tmp = await WhiteLightSource.cmd_turnLampOn.set_start(power=wls_power)
    for wave in np.linspace(300, 1200, 10):
        print(wave, Time.now())
        for entry_width in [0.5, 4.9]: #[, 0.1]: #, 3., 4., 4.9]:
            for exit_width in [0.5, 4.9]: #[0.02,0.04, 0.06, 0.08, 0.1]: #, 3., 4., 4.9]:
                start_meas = Time.now().isot
                wavelength, entry_width, exit_width, gratingType = await set_params(wave, entry_width, exit_width, grating)
                await elec_meas(exp_time)   
                spec_fn = await spect_meas(exp_time)
                await asyncio.sleep(5.)
                end_meas = Time.now().isot
                data.append([start_meas, end_meas, wls_power, exp_time, wavelength, entry_width, exit_width, gratingType, spec_fn])

In [ ]:
elec_data = []
for i, row in df.iterrows():
    try:
        filen_, mean_, median_, std_ = await get_step_data(Time(row['start']), Time(row['end']), 5)
        elec_data.append([row['start'], row['end'], filen_, mean_, median_, std_])
    except:
        elec_data.append([row['start'], row['end'], 'None', np.nan, np.nan, np.nan])
dd = pd.DataFrame(elec_data, columns = ['start','end','filen','mean','median','std'])
ee = [Time(e).isot for e in df['end']]
ss = [Time(e).isot for e in df['start']]
t = Table.from_pandas(dd)
t['end'] = ee
t['start'] = ss
    
spectra = []
for f in df['file']:
    fs_file = os.path.join('/home/parfa30/DATA/fiberSpec_files', f)
    hdu = fits.open(fs_file)
    wave = hdu[1].data['wavelength'][0].flatten()
    spectra.append(hdu[0].data)

waves = np.array(wave, dtype=[('Wavelength', ')])